In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import TimeSeriesSplit

## QUANTILE REGRESSION

In [2]:
data = pd.read_csv('non_nan.csv', )
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [3]:
def create_lags_cov(data, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)
    
    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs

In [4]:
def create_lags_returns(data, tickers, period):

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_tickers

In [5]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(data, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(data, tickers, period= period_returns)

In [6]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(data, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(data, tickers, period= period_returns)

# Creating dummy variables
data['Quater_dummy'] = 0
data['Quater_rippel'] = 0
data['Date'] = pd.to_datetime(data['Date'])
for index, row in data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_dummy'] = 1

In [7]:
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,NVO_lag1,NVO_lag2,NVS_lag1,NVS_lag2,PFE_lag1,PFE_lag2,ROG_lag1,ROG_lag2,Quater_dummy,Quater_rippel
0,2000-12-01,3.9,174.600,5.717143,-0.087123,-0.826852,2.331298,6.672740,5.420678,-0.380932,...,-9.882362,-4.279274,5.700307,-0.807726,2.604919,-3.860903,1.748252,13.043478,1,0
1,2001-01-01,4.2,175.600,5.240500,-0.066423,4.766324,-13.766945,-16.294179,-11.357522,-15.325723,...,-7.125809,-9.882362,13.379157,5.700307,4.026887,2.604919,12.886598,1.748252,1,0
2,2001-02-01,4.2,176.000,5.160952,-0.195020,-9.538746,4.143180,2.851534,4.509829,0.837527,...,12.824829,-7.125809,-4.290496,13.379157,-1.847877,4.026887,-5.022831,12.886598,0,0
3,2001-03-01,4.3,176.100,5.098947,-0.217255,-5.857225,5.615486,-6.323922,-9.828843,-3.186580,...,-0.350508,12.824829,-0.373563,-4.290496,-0.332146,-1.847877,-8.846156,-5.022831,1,0
4,2001-04-01,4.4,176.400,4.885455,0.756989,8.836250,-1.512969,-5.723911,10.300706,10.879156,...,-0.150755,-0.350508,-7.827520,-0.373563,-8.777164,-0.332146,-0.112520,-8.846156,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,8.619586,14.479974,1.295221,10.390734,3.090302,7.690670,9.446071,-53.665066,1,0
266,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,2.541749,8.619586,-0.110227,1.295221,-13.817335,3.090302,16.968326,9.446071,0,0
267,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,1.592445,2.541749,-7.172811,-0.110227,-7.286115,-13.817335,5.451681,16.968326,1,0
268,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,12.873250,1.592445,9.367574,-7.172811,0.566924,-7.286115,11.025813,5.451681,1,0


### Defining tick_loss function

In [8]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

In [9]:
data.columns

Index(['Date', 'unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth',
       'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE',
       'ROG', 'inflation_change', 'unemp_change', 'treasury_yield_change',
       'unemployment_rate_lag1', 'unemployment_rate_lag2',
       'unemployment_rate_lag3', 'CPI_lag1', 'CPI_lag2', 'CPI_lag3',
       'treasury_yield_lag1', 'treasury_yield_lag2', 'treasury_yield_lag3',
       'GDP_growth_lag1', 'GDP_growth_lag2', 'GDP_growth_lag3',
       'SP500_return_lag1', 'SP500_return_lag2', 'SP500_return_lag3',
       'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'AZN_lag1',
       'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1', 'JNJ_lag2', 'LLY_lag1',
       'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1', 'NVO_lag2', 'NVS_lag1',
       

In [10]:
# rolling_forecast_QR adaptable to all horizons

import statsmodels.api as sm
import numpy as np

def rolling_forecast_QR(data, dates, ticker, alpha, columns_to_exclude, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude,axis=1)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude, axis=1)
        X_test = X_test.iloc[0:horizon]
        y_test = test_df[ticker][0:horizon]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)
        all_var_forecast.append(VaR_forecast)

        r = y_test
        loss = tick_loss(alpha, r, VaR_forecast)
        t_loss += loss

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)

    return t_loss, all_var_forecast

# Naïve Models Specifications

## 1. Macros Only

We are only going to train the model with the lagged macroeconomic variables

In [11]:
data_macro = data.copy()
to_exclude = []

for t in tickers:
    to_exclude.append(t)
    to_exclude.append(f'{t}_lag1')
    to_exclude.append(f'{t}_lag2')

for covariate in covs:
    to_exclude.append(covariate)

to_exclude.append('Date')

to_exclude

['AZN',
 'AZN_lag1',
 'AZN_lag2',
 'BMY',
 'BMY_lag1',
 'BMY_lag2',
 'JNJ',
 'JNJ_lag1',
 'JNJ_lag2',
 'LLY',
 'LLY_lag1',
 'LLY_lag2',
 'MRK',
 'MRK_lag1',
 'MRK_lag2',
 'NVO',
 'NVO_lag1',
 'NVO_lag2',
 'NVS',
 'NVS_lag1',
 'NVS_lag2',
 'PFE',
 'PFE_lag1',
 'PFE_lag2',
 'ROG',
 'ROG_lag1',
 'ROG_lag2',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'Date']

### 1-step-ahead VaR: 1 month

In [12]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']

In [13]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=1)
    print(f'{t}-1m-loss: {loss} \n')
    print(f'{t}-1m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-1m-loss: 0.5973984308307704 

AZN-1m-VaR-forecast: [ -8.42148532  -4.7117599   -8.73083435 -10.29276875 -10.40801662
 -12.07501134  -0.88210064  -4.35815879 -14.94508848  -9.21300088
 -10.71829988 -10.77965598] 

BMY-1m-loss: 0.575476041176639 

BMY-1m-VaR-forecast: [ -6.32739385 -10.08606125 -10.11298188 -17.98095261   0.50303324
 -10.30913906 -27.15542053 -13.15869627 -13.89185267 -11.12614839
 -14.05645782 -15.24167195] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 0.5455903436256422 

JNJ-1m-VaR-forecast: [ -4.10273981  -1.50701229  -5.01958536  -1.08544395  -9.15235362
 -10.03262881  -3.81811447  -7.45731708  -9.21308055  -9.4632912
 -13.60501796 -12.25406794] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-1m-loss: 1.5483088342923867 

PFE-1m-VaR-forecast: [-13.24923352  -8.21720109  -0.65715548  -6.597352   -13.31371882
 -15.26685799  -6.74056392  -9.44136652 -17.66492282 -11.17945784
 -11.6885489  -15.34363344] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 4.889701702297191 

ROG-1m-VaR-forecast: [-11.68330142 -41.8838649  -19.14112647 -21.78093244 -16.30590638
  -6.99553474 -36.8828301  -18.41611147 -16.53053263 -20.68271407
 -23.75015203 -32.15856596] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


- AZN-1m-loss: 0.5973984308307704
- BMY-1m-loss: 0.575476041176639
- JNJ-1m-loss: 0.5455903436256422 
- LLY-1m-loss: 1.6973569041256713
- MRK-1m-loss: 0.8044542059407872
- NVO-1m-loss: 0.8365805401165188 
- NVS-1m-loss: 0.9885674254982467
- PFE-1m-loss: 1.5483088342
- ROG-1m-loss: 4.889701702297191 

### 3-step-ahead VaR: 3 month

In [14]:
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']

In [15]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=3)
    print(f'{t}-3m-loss: {loss} \n')
    print(f'{t}-3m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-3m-loss: 1.8657196198534352 

AZN-3m-VaR-forecast: [ -8.42148532  -4.76214774  -8.72119684  -4.7117599   -8.73083344
 -10.29276934  -8.73083435 -10.29276814 -10.23471045 -10.29276875
 -10.23470966 -12.0485803  -10.40801662 -12.07501151  -0.88210546
 -12.07501134  -0.88210552  -4.3960085   -0.88210064  -4.39600668
 -15.45365667  -4.35815879 -15.36598029  -9.27960177 -14.94508848
  -9.21299728 -10.78996759  -9.21300088 -10.78996868 -10.86165687] 

BMY-3m-loss: 1.78982145409515 

BMY-3m-VaR-forecast: [ -6.32739385 -10.08606957 -10.03766047 -10.08606125 -10.03766063
 -19.0498058  -10.11298188 -17.98089824   0.5045923  -17.98095261
   0.504554    -9.32897444   0.50303324 -10.30913866 -26.03678694
 -10.30913906 -26.03678678 -13.15870975 -27.15542053 -14.04993329
 -15.06373016 -13.15869627 -14.75169617 -12.4478426  -13.89185267
 -11.1261478  -14.18965913 -11.12614839 -14.18965906 -15.47249668] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-3m-loss: 1.7414492101651757 

JNJ-3m-VaR-forecast: [ -4.10273981  -1.73005601  -5.051173    -1.50701229  -4.89682511
  -0.45140973  -5.01958536  -1.03936663  -9.46584755  -1.08544395
  -9.3944079  -10.1800587   -9.15235362 -10.06405251  -3.92214042
 -10.03262881  -3.81812409  -7.45732275  -3.81811447  -7.45731819
  -9.21994637  -7.45731708  -9.21994745  -9.4056649   -9.21308055
  -9.46304361 -13.68997191  -9.4632912  -13.69024411 -12.79339859] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-3m-loss: 5.340884406757343 

PFE-3m-VaR-forecast: [-13.24923352  -8.21673445  -0.65617648  -8.21720109  -0.65715679
 -14.04799868  -0.65715548 -14.0479555  -13.39868611  -6.597352
 -13.26571618 -16.39951912 -13.31371882 -15.73575555  -7.00050793
 -15.26685799  -6.77834304  -9.59805536  -6.74056392  -9.60428107
 -20.89187025  -9.44136652 -20.88461641 -10.95062498 -17.66492282
 -11.10151429 -11.68853602 -11.17945784 -11.68093944 -15.40877533] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-3m-loss: 17.00619277443582 

ROG-3m-VaR-forecast: [-11.68330142 -42.65247597 -18.44401297 -41.8838649  -17.52279921
 -14.33725342 -19.14112647 -22.49417865 -15.5146311  -21.78093244
 -16.10492686  -4.37871854 -16.30590638  -7.13141312 -30.14298626
  -6.99553474 -30.97840641 -21.0347158  -36.8828301  -17.01389321
 -16.56134413 -18.41611147 -15.91827249 -20.8277197  -16.53053263
 -23.62313499 -23.66241338 -20.68271407 -23.71193805 -31.72183356] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


- AZN-3m-loss: 1.8657196198534352 
- BMY-3m-loss: 1.78982145409515 
- JNJ-3m-loss: 1.7414492101651757
- LLY-3m-loss: 5.1545043086404245 
- MRK-3m-loss: 2.417655156722561
- NVO-3m-loss: 2.6847909078247705
- NVS-3m-loss: 2.220851134172759
- PFE-3m-loss: 5.340884406757343
- ROG-3m-loss: 17.00619277443582

### 6-steap-ahead VaR: 6 months

In [16]:
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [17]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=6)
    print(f'{t}-6m-loss: {loss} \n')
    print(f'{t}-6m-VaR-forecast: {var_forecast} \n')

AZN-6m-loss: 3.857545776417826 

AZN-6m-VaR-forecast: [ -8.42148532  -4.76214774  -8.72119684 -10.21344803 -10.23678806
 -12.14925975  -4.7117599   -8.73083344 -10.29276934 -10.23470969
 -12.04858076  -0.36909286  -8.73083435 -10.29276814 -10.23471045
 -12.04858614  -0.36909556  -3.34354967 -10.29276875 -10.23470966
 -12.0485803   -0.36909145  -3.34355008 -14.81829204 -10.40801662
 -12.07501151  -0.88210546  -4.39600877 -15.45366125  -9.29345746
 -12.07501134  -0.88210552  -4.3960085  -15.45366091  -9.29345784
 -10.96601935  -0.88210064  -4.39600668 -15.45365667  -9.29345776
 -10.96601823 -10.86158297] 

BMY-6m-loss: 3.8596086975274466 

BMY-6m-VaR-forecast: [ -6.32739385 -10.08606957 -10.03766047 -19.04982591   0.08663479
  -8.98235004 -10.08606125 -10.03766063 -19.0498058    0.08664263
  -8.98235678 -27.85599974 -10.11298188 -17.98089824   0.5045923
  -9.32897999 -27.15543983 -14.0499484  -17.98095261   0.504554
  -9.32897444 -27.15539758 -14.04996388 -15.06370721   0.50303324
 -10.3

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-6m-loss: 8.53842655194998 

LLY-6m-VaR-forecast: [ -9.72316526 -10.92657332   1.4783368   -9.18192666 -17.60685583
  -6.0059175  -10.89561498   1.52217345  -9.18612136 -17.57869723
  -5.99363424  -0.5305359    3.55990219  -9.37171769 -16.27838195
  -5.42433641   0.16572756 -16.51193239  -9.51998078 -17.07439582
  -9.83958311  -3.81626073 -15.09901245 -11.64083651 -17.05814377
  -9.8563554   -3.86366391 -15.08133477 -11.64493683 -17.71936595
  -9.93714511  -4.09141798 -14.99617926 -11.66480621 -17.76048407
 -11.28807679  -3.81632016 -15.09901427 -11.64082227 -17.71079145
 -11.19109264 -14.90980049] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-6m-loss: 4.978882489437742 

MRK-6m-VaR-forecast: [-12.61091981 -14.54885177 -10.28937089 -22.77097605  -7.37735049
  -9.14040679 -14.63560736 -12.35471301 -20.74914834  -7.24373011
 -10.0100501  -14.05383327 -12.35470924 -20.74915475  -7.24372807
 -10.01004504 -14.05382905 -12.68524332 -22.77105978  -7.37728907
  -9.1403012  -11.50708333 -14.43375841 -18.91049929  -7.24374042
 -10.01007897 -14.05385359 -12.68521641 -17.173883   -13.64527944
 -10.0100394  -14.05383002 -12.68524843 -17.17389467 -13.64525024
 -13.90047169 -12.43103572 -15.04349008 -18.06186823 -11.61118081
 -13.62369703 -18.90477262] 



...

NVO-6m-loss: 5.695354265514267 

NVO-6m-VaR-forecast: [-15.83820621 -16.11355647 -24.49281326 -15.66661031 -17.97219201
  -9.39063129 -15.80375255 -24.31753692 -17.14410667 -17.02476617
  -7.36194224 -10.52509995 -24.31753896 -17.14410574 -17.02476794
  -7.36194362 -10.52510112 -18.52195358 -17.65521848 -16.88865303
  -7.18746458 -10.35564153 -18.98515238 -13.3586004  -17.68289112
  -7.24796626  -8.75942712 -17.36098637 -14.05774756 -13.56893124
  -7.24699642  -8.7591407  -17.36123673 -14.05728274 -13.56855531
 -10.12555563  -7.65213956 -18.46785083 -12.23350422 -11.97953805
  -6.52093265  -8.36614307] 

NVS-6m-loss: 4.1967392406958455 

NVS-6m-VaR-forecast: [ -1.17385893  -3.95129141  -7.59904316  -5.02421032 -15.7018874
  -5.75078453  -5.79711045  -5.65633631 -10.69348379 -14.44650923
  -8.77379734  -3.43381957  -5.65635331 -10.69347786 -14.44649901
  -8.77384708  -3.43381319  -9.93235083 -10.51611927 -14.25829862
  -9.82735936  -3.71101069  -9.29177787  -8.02675412 -14.25829584
  -9

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-6m-loss: 9.425849982841536 

PFE-6m-VaR-forecast: [-13.24923352  -8.21673445  -0.65617648 -14.04631325 -13.39860928
  -8.92786397  -8.21720109  -0.65715679 -14.04799868 -13.39866622
  -8.92156481  -5.48135596  -0.65715548 -14.0479555  -13.39868611
  -8.92156755  -5.4813247  -12.34505161  -6.597352   -13.26571618
 -16.39951912  -6.99836535  -9.62336628 -21.11246763 -13.31371882
 -15.73575555  -7.00050793  -9.71083371 -21.17775922 -11.24002033
 -15.26685799  -6.77834304  -9.59805536 -21.05775637 -11.08382463
 -13.01609104  -6.74056392  -9.60428107 -20.89187025 -11.06073498
 -12.91944744 -16.66076959] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-6m-loss: 42.736614702794 

ROG-6m-VaR-forecast: [-11.68330142 -42.65247597 -18.44401297 -13.77867301 -19.53927276
 -15.1593173  -41.8838649  -17.52279921 -14.33725342 -19.54279639
 -14.70032894 -30.86377096 -19.14112647 -22.49417865 -15.5146311
  -8.40652571 -35.85265367 -21.45965092 -21.78093244 -16.10492686
  -4.37871854 -35.08148385 -19.83055405  -6.62286436 -16.30590638
  -7.13141312 -30.14298626 -19.64002819 -13.33177939 -24.3143873
  -6.99553474 -30.97840641 -21.0347158  -12.77436837 -22.86825066
 -16.52782388 -36.8828301  -17.01389321 -16.56134413 -20.81255943
 -23.8629035  -29.56504793] 



- AZN-6m-loss: 3.857545776417826
- BMY-6m-loss: 3.8596086975274466
- JNJ-6m-loss: 3.07118173817229 
- LLY-6m-loss: 8.53842655194998
- MRK-6m-loss: 4.978882489437742
- NVO-6m-loss: 5.695354265514267
- NVS-6m-loss: 4.1967392406958455
- PFE-6m-loss: 9.425849982841536 
- ROG-6m-loss: 42.736614702794 

### 9-step-ahead VaR: 9 month

In [18]:
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']

In [19]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=9)
    print(f'{t}-9m-loss: {loss} \n')
    print(f'{t}-9m-VaR-forecast: {var_forecast} \n')

AZN-9m-loss: 9.114092203363686 

AZN-9m-VaR-forecast: [ -8.42148532  -4.76214774  -8.72119684 -10.21344803 -10.23678806
 -12.14925975  -0.35600183  -3.23348072 -14.78150875  -4.7117599
  -8.73083344 -10.29276934 -10.23470969 -12.04858076  -0.36909286
  -3.34355078 -14.81829265  -9.54888288  -8.73083435 -10.29276814
 -10.23471045 -12.04858614  -0.36909556  -3.34354967 -14.8182937
  -9.54888244 -11.04618244 -10.29276875 -10.23470966 -12.0485803
  -0.36909145  -3.34355008 -14.81829204  -9.54888201 -11.04618345
 -10.75959376 -10.40801662 -12.07501151  -0.88210546  -4.39600877
 -15.45366125  -9.29345746 -10.96601925 -10.86158226 -12.07501134
  -0.88210552  -4.3960085  -15.45366091  -9.29345784 -10.96601935
 -10.86158254  -0.88210064  -4.39600668 -15.45365667  -9.29345776
 -10.96601823 -10.86158297] 

BMY-9m-loss: 9.254001509976497 

BMY-9m-VaR-forecast: [ -6.32739385 -10.08606957 -10.03766047 -19.04982591   0.08663479
  -8.98235004 -27.85601537 -14.60837053 -15.31386633 -10.08606125
 -10.03

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-9m-loss: 18.365013537311157 

LLY-9m-VaR-forecast: [ -9.72316526 -10.92657332   1.4783368   -9.18192666 -17.60685583
  -6.0059175   -0.54547222 -16.2580248  -13.98769522 -10.89561498
   1.52217345  -9.18612136 -17.57869723  -5.99363424  -0.5305359
 -16.26340351 -13.97614684 -14.39134296   3.55990219  -9.37171769
 -16.27838195  -5.42433641   0.16572756 -16.51193239 -13.44174034
 -12.89984554 -13.66404456  -9.51998078 -17.07439582  -9.83958311
  -3.81626073 -15.09901245 -11.64083651 -17.71074858 -11.19109781
 -14.90978064 -17.05814377  -9.8563554   -3.86366391 -15.08133477
 -11.64493683 -17.71936595 -11.20776111 -14.91559703  -9.93714511
  -4.09141798 -14.99617926 -11.66480621 -17.76048407 -11.28807679
 -14.94341427  -3.81632016 -15.09901427 -11.64082227 -17.71079145
 -11.19109264 -14.90980049] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-9m-loss: 11.636008464094882 

MRK-9m-VaR-forecast: [-12.61091981 -14.54885177 -10.28937089 -22.77097605  -7.37735049
  -9.14040679 -11.50705575 -14.43373401 -18.91052612 -14.63560736
 -12.35471301 -20.74914834  -7.24373011 -10.0100501  -14.05383327
 -12.68523899 -17.17389513 -13.64525356 -12.35470924 -20.74915475
  -7.24372807 -10.01004504 -14.05382905 -12.68524332 -17.17389792
 -13.64524833 -13.90046522 -22.77105978  -7.37728907  -9.1403012
 -11.50708333 -14.43375841 -18.91049929 -11.76627589 -13.70929911
 -18.53622121  -7.24374042 -10.01007897 -14.05385359 -12.68521641
 -17.173883   -13.64527944 -13.90047635 -17.65857445 -10.0100394
 -14.05383002 -12.68524843 -17.17389467 -13.64525024 -13.90047169
 -17.65859335 -12.43103572 -15.04349008 -18.06186823 -11.61118081
 -13.62369703 -18.90477262] 



...

NVO-9m-loss: 13.025934221707637 

NVO-9m-VaR-forecast: [-15.83820621 -16.11355647 -24.49281326 -15.66661031 -17.97219201
  -9.39063129 -10.19622641 -17.30294231 -14.32630077 -15.80375255
 -24.31753692 -17.14410667 -17.02476617  -7.36194224 -10.52509995
 -18.52195448 -12.89166179 -15.06558568 -24.31753896 -17.14410574
 -17.02476794  -7.36194362 -10.52510112 -18.52195358 -12.89166321
 -15.06558516 -10.78482138 -17.65521848 -16.88865303  -7.18746458
 -10.35564153 -18.98515238 -13.3586004  -14.49028471 -10.61969227
  -8.06976338 -17.68289112  -7.24796626  -8.75942712 -17.36098637
 -14.05774756 -13.56893124 -10.12640848  -5.83232213  -7.24699642
  -8.7591407  -17.36123673 -14.05728274 -13.56855531 -10.12555563
  -5.83282179  -7.65213956 -18.46785083 -12.23350422 -11.97953805
  -6.52093265  -8.36614307] 

NVS-9m-loss: 9.482837515999858 

NVS-9m-VaR-forecast: [ -1.17385893  -3.95129141  -7.59904316  -5.02421032 -15.7018874
  -5.75078453  -4.66649463 -10.33336141  -7.87174883  -5.79711045
  -5

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-9m-loss: 18.990352524271913 

PFE-9m-VaR-forecast: [-13.24923352  -8.21673445  -0.65617648 -14.04631325 -13.39860928
  -8.92786397  -5.48120417 -12.34146163 -18.30801982  -8.21720109
  -0.65715679 -14.04799868 -13.39866622  -8.92156481  -5.48135596
 -12.34505264 -18.30619939  -9.88211087  -0.65715548 -14.0479555
 -13.39868611  -8.92156755  -5.4813247  -12.34505161 -18.30620044
  -9.88208701 -11.96740027  -6.597352   -13.26571618 -16.39951912
  -6.99836535  -9.62336628 -21.11246763 -11.24735789 -13.28333144
 -17.52909263 -13.31371882 -15.73575555  -7.00050793  -9.71083371
 -21.17775922 -11.24002033 -13.16544837 -17.23574103 -15.26685799
  -6.77834304  -9.59805536 -21.05775637 -11.08382463 -13.01609104
 -16.82269253  -6.74056392  -9.60428107 -20.89187025 -11.06073498
 -12.91944744 -16.66076959] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-9m-loss: 79.82807832015772 

ROG-9m-VaR-forecast: [-11.68330142 -42.65247597 -18.44401297 -13.77867301 -19.53927276
 -15.1593173  -30.63266207 -23.97444639 -19.73177042 -41.8838649
 -17.52279921 -14.33725342 -19.54279639 -14.70032894 -30.86377096
 -23.82979098 -19.42725771 -22.50836628 -19.14112647 -22.49417865
 -15.5146311   -8.40652571 -35.85265367 -21.45965092  -8.67219057
 -24.52819644 -18.70140862 -21.78093244 -16.10492686  -4.37871854
 -35.08148385 -19.83055405  -6.62286436 -24.81740371 -19.14120039
 -19.20429064 -16.30590638  -7.13141312 -30.14298626 -19.64002819
 -13.33177939 -24.3143873  -18.45081013 -24.25502326  -6.99553474
 -30.97840641 -21.0347158  -12.77436837 -22.86825066 -16.52782388
 -23.46906607 -36.8828301  -17.01389321 -16.56134413 -20.81255943
 -23.8629035  -29.56504793] 



- AZN-9m-loss: 9.114092203363686
- BMY-9m-loss: 9.254001509976497
- JNJ-9m-loss: 7.0102317895879 
- LLY-9m-loss: 18.365013537311157
- MRK-9m-loss: 11.636008464094882
- NVO-9m-loss: 13.025934221707637 
- NVS-9m-loss: 9.482837515999858
- PFE-9m-loss: 18.990352524271913 
- ROG-9m-loss: 79.82807832015772

### 12-step-ahead VaR: 12 months

In [20]:
dates12m = ['2022-05-01']

In [21]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=12)
    print(f'{t}-12m-loss: {loss} \n')
    print(f'{t}-12m-VaR-forecast: {var_forecast} \n')

AZN-12m-loss: 7.520407923044603 

AZN-12m-VaR-forecast: [ -8.42148532  -4.76214774  -8.72119684 -10.21344803 -10.23678806
 -12.14925975  -0.35600183  -3.23348072 -14.78150875  -9.4753878
 -10.94301546 -10.67661861] 

BMY-12m-loss: 7.410233048427555 

BMY-12m-VaR-forecast: [ -6.32739385 -10.08606957 -10.03766047 -19.04982591   0.08663479
  -8.98235004 -27.85601537 -14.60837053 -15.31386633 -13.12366045
 -15.26709152 -18.46848488] 

JNJ-12m-loss: 6.182844572027813 

JNJ-12m-VaR-forecast: [ -4.10273981  -1.73005601  -5.051173    -1.1907015   -9.5421583
  -8.67691582  -2.46840667  -7.5009079   -8.98417368  -9.29412923
 -13.1172061  -12.58810601] 

LLY-12m-loss: 18.926479661664928 

LLY-12m-VaR-forecast: [ -9.72316526 -10.92657332   1.4783368   -9.18192666 -17.60685583
  -6.0059175   -0.54547222 -16.2580248  -13.98769522 -14.42345577
 -13.92169439 -16.5547594 ] 

MRK-12m-loss: 9.466614858117968 

MRK-12m-VaR-forecast: [-12.61091981 -14.54885177 -10.28937089 -22.77097605  -7.37735049
  -9.14

- AZN-12m-loss: 7.520407923044603
- BMY-12m-loss: 7.410233048427555 
- JNJ-12m-loss: 6.182844572027813
- LLY-12m-loss: 18.926479661664928
- MRK-12m-loss: 9.466614858117968
- NVO-12m-loss: 10.750503619065153 
- NVS-12m-loss: 11.536438011396003 
- PFE-12m-loss: 16.002290711471506
- ROG-12m-loss: 50.22000803618026 

# 2. Ticker data

In [22]:
data_stocks = data.copy()
to_exclude2 = []

for covariate in covs:
    to_exclude2.append(covariate)
    to_exclude2.append(f'{covariate}_lag1')
    to_exclude2.append(f'{covariate}_lag2')
    to_exclude2.append(f'{covariate}_lag3')

for t in tickers:
    to_exclude2.append(t)

to_exclude2.append('Date')
to_exclude2

['unemployment_rate',
 'unemployment_rate_lag1',
 'unemployment_rate_lag2',
 'unemployment_rate_lag3',
 'CPI',
 'CPI_lag1',
 'CPI_lag2',
 'CPI_lag3',
 'treasury_yield',
 'treasury_yield_lag1',
 'treasury_yield_lag2',
 'treasury_yield_lag3',
 'GDP_growth',
 'GDP_growth_lag1',
 'GDP_growth_lag2',
 'GDP_growth_lag3',
 'SP500_return',
 'SP500_return_lag1',
 'SP500_return_lag2',
 'SP500_return_lag3',
 'inflation_change',
 'inflation_change_lag1',
 'inflation_change_lag2',
 'inflation_change_lag3',
 'unemp_change',
 'unemp_change_lag1',
 'unemp_change_lag2',
 'unemp_change_lag3',
 'treasury_yield_change',
 'treasury_yield_change_lag1',
 'treasury_yield_change_lag2',
 'treasury_yield_change_lag3',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG',
 'Date']

### 1-step-ahead VaR: 1 month

In [23]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=1)
    print(f'{t}-1m-loss: {loss} \n')
    print(f'{t}-1m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-1m-loss: 0.9246658669637693 

AZN-1m-VaR-forecast: [ -9.36771076  -8.13137597  -2.62882678  -9.59354468  -1.72839603
  -5.33447889 -17.55886985 -20.07621851  -2.30508953  -8.09234079
  -9.79748238 -11.79650003] 

BMY-1m-loss: 1.8087503597175576 

BMY-1m-VaR-forecast: [-13.82705149 -13.47864121  -2.25699303  -8.89940481  -6.34795734
  -9.13750069 -27.16562517 -18.5975089    5.13173218  -9.56104948
 -15.84115568  -8.21397313] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 1.4361016556024417 

JNJ-1m-VaR-forecast: [ -6.15649329  -7.81781753  -0.07232725  -4.33346428  -2.70630701
  -4.49657262 -14.93481871 -11.78481826   0.84192782  -0.29090706
  -4.72426797  -8.02053309] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

NVO-1m-loss: 1.377840118704824 

NVO-1m-VaR-forecast: [ -8.20196364 -12.41155919  -0.09273882  -9.0359561   -3.89089468
  -4.23598953 -20.74529059 -19.09984813   2.30348934  -9.1813527
 -11.11360099  -8.83932217] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-1m-loss: 1.791568422036246 

NVS-1m-VaR-forecast: [ -5.78553099 -11.63144006  -1.73271593  -7.9543678   -3.1623979
  -6.09390647 -17.7371067  -20.98813932   3.32509782  -5.01667869
  -4.84942     -5.92433106] 

PFE-1m-loss: 1.3525566069687258 

PFE-1m-VaR-forecast: [ -2.59299749 -11.40880616  -4.79890379 -12.26902203  -1.55699875
  -2.91433986 -14.63434576 -11.79033581  -1.06547875  -9.03582999
 -10.30013641  -5.41391124] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 5.831112464847229 

ROG-1m-VaR-forecast: [-18.52425012 -20.56218378   0.50279445 -21.71270189  -5.49670467
   1.58608746 -34.32139633 -39.97136489   4.11646439 -17.58537641
 -12.87910199  -6.4869154 ] 



### 3-step-ahead VaR: 3 months

In [24]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=3)
    print(f'{t}-1m-loss: {loss} \n')
    print(f'{t}-1m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-1m-loss: 3.096447107630346 

AZN-1m-VaR-forecast: [ -9.36771076  -8.11676319  -2.62579691  -8.13137597  -2.71009107
  -9.10494199  -2.62882678  -9.13988034  -0.93597553  -9.59354468
  -0.62399616  -5.58330016  -1.72839603  -5.18649739 -17.55886023
  -5.33447889 -17.52098174 -20.3700372  -17.55886985 -20.51161758
  -1.8040118  -20.07621851  -2.30506822  -8.46628317  -2.30508953
  -8.4663028  -11.25920174  -8.09234079 -10.74712811 -11.74367257] 

BMY-1m-loss: 6.177991979688931 

BMY-1m-VaR-forecast: [-13.82705149 -13.4786425   -2.25700077 -13.47864121  -2.25700055
  -8.82525446  -2.25699303  -8.82522056  -7.23641161  -8.89940481
  -6.16793858  -9.13750884  -6.34795734  -9.12674616 -26.83015557
  -9.13750069 -27.19979926 -18.68694508 -27.16562517 -18.73067357
   5.08063517 -18.5975089    5.13178423  -8.76117011   5.13173218
  -8.76141521 -15.57760286  -9.56104948 -16.53572245  -8.11929194] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 4.970245290854381 

JNJ-1m-VaR-forecast: [ -6.15649329  -7.86964367  -0.14101453  -7.81781753  -0.07340876
  -4.00853479  -0.07232725  -4.00773969  -2.52442805  -4.33346428
  -2.70656917  -4.4927618   -2.70630701  -4.49657374 -14.93480162
  -4.49657262 -14.93480655 -11.78481852 -14.93481871 -11.78480852
   0.80293103 -11.78481826   0.80294235   0.84603873   0.84192782
   0.64352664  -3.78793344  -0.29090706  -4.73634915  -7.95273913] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

NVO-1m-loss: 4.631828819819036 

NVO-1m-VaR-forecast: [ -8.20196364 -12.41152516  -0.09280017 -12.41155919  -0.09279126
  -9.03595952  -0.09273882  -9.03587345  -3.89077088  -9.0359561
  -3.89088324  -4.23598732  -3.89089468  -4.23598618 -20.74518312
  -4.23598953 -20.74527259 -17.69442865 -20.74529059 -17.69440387
   1.99892731 -19.09984813   2.69477652  -8.61886289   2.30348934
  -8.24504154  -9.62869152  -9.1813527  -11.11361083  -8.83932106] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-1m-loss: 5.94593699482729 

NVS-1m-VaR-forecast: [ -5.78553099 -10.93243965  -2.05368315 -11.63144006  -1.7261771
  -7.57732382  -1.73271593  -7.56679896  -3.7224835   -7.9543678
  -3.2205499   -6.34823796  -3.1623979   -6.13551306 -17.73712234
  -6.09390647 -17.76466108 -20.97157367 -17.7371067  -20.92396468
   3.32509892 -20.98813932   3.32097305  -3.95435462   3.32509782
  -4.26497817  -3.9456217   -5.01667869  -4.84942552  -5.92438122] 

PFE-1m-loss: 4.606524350287815 

PFE-1m-VaR-forecast: [ -2.59299749 -11.40885662  -4.85362477 -11.40880616  -4.85358165
 -12.0208297   -4.79890379 -12.023343    -1.24671363 -12.26902203
  -1.53808349  -2.90407094  -1.55699875  -2.97292984 -14.73048589
  -2.91433986 -14.63837266 -12.14804509 -14.63434576 -12.14742925
  -1.29177635 -11.79033581  -1.47654692 -10.41976804  -1.06547875
  -8.75615288  -9.32157885  -9.03582999 -10.30013778  -5.41391359] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 20.502695782956835 

ROG-1m-VaR-forecast: [-18.52425012 -20.56215673   0.50279267 -20.56218378   0.50274662
 -20.03580684   0.50279445 -20.03573742  -3.20141294 -21.71270189
  -5.49610395   1.75055126  -5.49670467   1.75028542 -21.7619532
   1.58608746 -22.71407325 -43.55680943 -34.32139633 -39.97139764
   4.11643652 -39.97136489   4.116463   -17.5853904    4.11646439
 -17.58538471 -12.87903768 -17.58537641 -12.87906227  -6.48683556] 



### 6-step-ahead VaR: 6 months

In [25]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=6)
    print(f'{t}-6m-loss: {loss} \n')
    print(f'{t}-6m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-6m-loss: 5.945866171920068 

AZN-6m-VaR-forecast: [ -9.36771076  -8.11676319  -2.62579691  -9.14115752  -0.93883327
  -5.31214763  -8.13137597  -2.71009107  -9.10494199  -0.85959429
  -5.4433708  -17.4146239   -2.62882678  -9.13988034  -0.93597553
  -5.31685572 -17.44209476 -21.82716361  -9.59354468  -0.62399616
  -5.58330016 -18.79505434 -20.67081221  -0.15284737  -1.72839603
  -5.18649739 -17.55886023 -20.5116112   -1.80401641  -8.09146432
  -5.33447889 -17.52098174 -20.3700372   -2.07022276  -8.32665206
 -11.1052869  -17.55886985 -20.51161758  -1.8040118   -8.09145845
 -10.745958   -11.74371174] 

BMY-6m-loss: 11.40577719057356 

BMY-6m-VaR-forecast: [-13.82705149 -13.4786425   -2.25700077  -8.82525597  -7.23638821
 -10.9645861  -13.47864121  -2.25700055  -8.82525446  -7.23638924
 -10.96458629 -29.07099176  -2.25699303  -8.82522056  -7.23641161
 -10.96459352 -29.07096965 -18.36168434  -8.89940481  -6.16793858
  -9.13750884 -27.19993263 -18.6869401    5.06959038  -6.34795734
  -9

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-6m-loss: 8.718092404130447 

JNJ-6m-VaR-forecast: [ -6.15649329  -7.86964367  -0.14101453  -4.08046094  -2.58067595
  -4.35494076  -7.81781753  -0.07340876  -4.00853479  -2.52356565
  -4.30056387 -14.06263368  -0.07232725  -4.00773969  -2.52442805
  -4.30017377 -14.05710323 -12.63789673  -4.33346428  -2.70656917
  -4.4927618  -14.9431869  -11.7704947    0.79542515  -2.70630701
  -4.49657374 -14.93480162 -11.78481607   0.80294172   0.84603414
  -4.49657262 -14.93480655 -11.78481852   0.80294329   0.84603558
  -3.86031112 -14.93481871 -11.78480852   0.80293103   0.84600017
  -3.86031313  -7.93130369] 

LLY-6m-loss: 11.09032853591484 

LLY-6m-VaR-forecast: [ -6.14072326  -9.61406506  -0.71319966  -7.07701567  -4.75988981
  -3.63485244  -9.61406632  -0.71319925  -7.07702452  -4.7598749
  -3.63483593 -12.48745292  -0.71319375  -7.0769978   -4.75988222
  -3.63485771 -12.48745352  -9.85612078  -7.99808656  -4.22116208
  -2.85236963 -12.18595047  -8.8378807   -0.26778537  -4.22119513
  -2.

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-6m-loss: 8.07677146390158 

NVO-6m-VaR-forecast: [ -8.20196364 -12.41152516  -0.09280017  -9.0359656   -3.89081498
  -4.23599235 -12.41155919  -0.09279126  -9.03595952  -3.89077751
  -4.23596671 -20.74534442  -0.09273882  -9.03587345  -3.89077088
  -4.23574211 -20.74496336 -17.69400808  -9.0359561   -3.89088324
  -4.23598732 -20.74520422 -17.69439936   1.99899317  -3.89089468
  -4.23598618 -20.74518312 -17.69439058   1.99900879  -9.10366992
  -4.23598953 -20.74527259 -17.69442865   1.99894193  -9.10367152
 -11.04063568 -20.74529059 -17.69440387   1.99892731  -9.10366849
 -11.04065224  -8.87532912] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-6m-loss: 11.184685945148818 

NVS-6m-VaR-forecast: [ -5.78553099 -10.93243965  -2.05368315  -7.39815503  -3.94794404
  -6.41649707 -11.63144006  -1.7261771   -7.57732382  -3.71704556
  -7.10465989 -18.42481041  -1.73271593  -7.56679896  -3.7224835
  -7.10693964 -18.41564165 -21.41956396  -7.9543678   -3.2205499
  -6.34823796 -18.0368459  -21.03564595   3.4154374   -3.1623979
  -6.13551306 -17.73712234 -20.92397172   3.32510661  -4.2649676
  -6.09390647 -17.76466108 -20.97157367   3.31595614  -4.29931743
  -3.98393014 -17.7371067  -20.92396468   3.32509892  -4.26497945
  -3.94562316  -5.86167016] 

PFE-6m-loss: 8.715167969773534 

PFE-6m-VaR-forecast: [ -2.59299749 -11.40885662  -4.85362477 -12.02079769  -1.19585113
  -2.86981911 -11.40880616  -4.85358165 -12.0208297   -1.19585955
  -2.86975133 -13.34911894  -4.79890379 -12.023343    -1.24671363
  -2.79976288 -12.73050619 -10.89045617 -12.26902203  -1.53808349
  -2.90407094 -14.61612651 -12.13545528  -1.31295692  -1.55699875
  -2.97

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


### 9-step-ahead VaR: 9 months

In [26]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=9)
    print(f'{t}-9m-loss: {loss} \n')
    print(f'{t}-9m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-9m-loss: 10.13261383769435 

AZN-9m-VaR-forecast: [ -9.36771076  -8.11676319  -2.62579691  -9.14115752  -0.93883327
  -5.31214763 -17.44306261 -21.8431454    1.43893377  -8.13137597
  -2.71009107  -9.10494199  -0.85959429  -5.4433708  -17.4146239
 -21.39722965   1.24786366  -5.17912081  -2.62882678  -9.13988034
  -0.93597553  -5.31685572 -17.44209476 -21.82716361   1.43208559
  -5.02445943  -9.16301588  -9.59354468  -0.62399616  -5.58330016
 -18.79505434 -20.67081221  -0.15284737  -5.02256714  -9.52811674
 -12.25969144] 

BMY-9m-loss: 19.361702901683017 

BMY-9m-VaR-forecast: [-13.82705149 -13.4786425   -2.25700077  -8.82525597  -7.23638821
 -10.9645861  -29.07099286 -18.36167234   5.83604035 -13.47864121
  -2.25700055  -8.82525446  -7.23638924 -10.96458629 -29.07099176
 -18.36167321   5.83604105 -10.69317517  -2.25699303  -8.82522056
  -7.23641161 -10.96459352 -29.07096965 -18.36168434   5.83606027
 -10.69310457 -16.13289529  -8.89940481  -6.16793858  -9.13750884
 -27.19993263 -18

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-9m-loss: 14.696143004454655 

JNJ-9m-VaR-forecast: [ -6.15649329  -7.86964367  -0.14101453  -4.08046094  -2.58067595
  -4.35494076 -14.23065566 -12.38943153   0.95332344  -7.81781753
  -0.07340876  -4.00853479  -2.52356565  -4.30056387 -14.06263368
 -12.63811302   0.9859846    0.29439777  -0.07232725  -4.00773969
  -2.52442805  -4.30017377 -14.05710323 -12.63789673   0.98662342
   0.29283701  -4.06512847  -4.33346428  -2.70656917  -4.4927618
 -14.9431869  -11.7704947    0.79542515   0.81719087  -3.85944181
  -7.92985768] 

LLY-9m-loss: 19.32721842346639 

LLY-9m-VaR-forecast: [ -6.14072326  -9.61406506  -0.71319966  -7.07701567  -4.75988981
  -3.63485244 -12.4874342   -9.85609075   0.50500719  -9.61406632
  -0.71319925  -7.07702452  -4.7598749   -3.63483593 -12.48745292
  -9.85610284   0.5050121   -2.8189522   -0.71319375  -7.0769978
  -4.75988222  -3.63485771 -12.48745352  -9.85612078   0.50502044
  -2.81891386  -6.37486071  -7.99808656  -4.22116208  -2.85236963
 -12.18595047  -8.

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-9m-loss: 12.515121581661338 

NVO-9m-VaR-forecast: [ -8.20196364 -12.41152516  -0.09280017  -9.0359656   -3.89081498
  -4.23599235 -20.74532754 -17.69445086   1.9989022  -12.41155919
  -0.09279126  -9.03595952  -3.89077751  -4.23596671 -20.74534442
 -17.69442666   1.99885486  -9.1036492   -0.09273882  -9.03587345
  -3.89077088  -4.23574211 -20.74496336 -17.69400808   1.99886087
  -9.10345328 -11.04047992  -9.0359561   -3.89088324  -4.23598732
 -20.74520422 -17.69439936   1.99899317  -9.10367064 -11.04056988
  -8.87517304] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-9m-loss: 18.39853240319071 

NVS-9m-VaR-forecast: [ -5.78553099 -10.93243965  -2.05368315  -7.39815503  -3.94794404
  -6.41649707 -16.68408759 -20.62413543   3.11285351 -11.63144006
  -1.7261771   -7.57732382  -3.71704556  -7.10465989 -18.42481041
 -21.41728061   3.76070118  -3.73065678  -1.73271593  -7.56679896
  -3.7224835   -7.10693964 -18.41564165 -21.41956396   3.75972716
  -3.719213    -3.40561605  -7.9543678   -3.2205499   -6.34823796
 -18.0368459  -21.03564595   3.4154374   -4.1629805   -3.88611927
  -5.97073062] 

PFE-9m-loss: 15.069090847440615 

PFE-9m-VaR-forecast: [ -2.59299749 -11.40885662  -4.85362477 -12.02079769  -1.19585113
  -2.86981911 -13.34953674 -11.03234821  -0.64736341 -11.40880616
  -4.85358165 -12.0208297   -1.19585955  -2.86975133 -13.34911894
 -11.03225199  -0.64733848 -10.29763781  -4.79890379 -12.023343
  -1.24671363  -2.79976288 -12.73050619 -10.89045617  -0.39856015
 -10.65368192  -8.29919839 -12.26902203  -1.53808349  -2.90407094
 -14.61612651 -12.

### 12-step-ahead VaR: 12 months

In [27]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=12)
    print(f'{t}-12m-loss: {loss} \n')
    print(f'{t}-12m-VaR-forecast: {var_forecast} \n')

AZN-12m-loss: 12.894688327558503 

AZN-12m-VaR-forecast: [ -9.36771076  -8.11676319  -2.62579691  -9.14115752  -0.93883327
  -5.31214763 -17.44306261 -21.8431454    1.43893377  -5.0186897
  -9.16117273 -11.15526038] 

BMY-12m-loss: 22.513338462580236 

BMY-12m-VaR-forecast: [-13.82705149 -13.4786425   -2.25700077  -8.82525597  -7.23638821
 -10.9645861  -29.07099286 -18.36167234   5.83604035 -10.69317891
 -16.1329343   -5.26414003] 

JNJ-12m-loss: 17.174892536981957 

JNJ-12m-VaR-forecast: [ -6.15649329  -7.86964367  -0.14101453  -4.08046094  -2.58067595
  -4.35494076 -14.23065566 -12.38943153   0.95332344   0.35792056
  -4.00051218  -7.98310128] 

LLY-12m-loss: 23.38840515001542 

LLY-12m-VaR-forecast: [ -6.14072326  -9.61406506  -0.71319966  -7.07701567  -4.75988981
  -3.63485244 -12.4874342   -9.85609075   0.50500719  -2.81894542
  -6.37486867  -8.11616972] 

MRK-12m-loss: 11.606960164957968 

MRK-12m-VaR-forecast: [ -7.32187049 -11.64091718  -1.81313379 -10.45349029  -2.63655592
  -

# 3. All Covariates

In [28]:
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

to_exclude3

['Date',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG']

### 1-step-ahead VaR: 1 months

In [29]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=1)
    print(f'{t}-1m-loss: {loss} \n')
    print(f'{t}-1m-VaR-forecast: {var_forecast} \n')

AZN-1m-loss: 0.5079276964925706 

AZN-1m-VaR-forecast: [ -7.61857227  -0.37745581  -6.82325599 -11.48124416 -12.30741192
  -5.26026587  -1.93132624 -14.50824726 -17.25928955  -1.25552064
  -8.882634   -19.8281017 ] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-1m-loss: 0.5918539103580055 

BMY-1m-VaR-forecast: [-10.35690661 -12.32950473 -10.04379576   2.70476258 -14.22136722
 -18.03919733 -20.05837988 -13.32710257  -3.68305233 -13.64784223
 -19.39644759  -7.57230608] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 2.1903887543716873 

JNJ-1m-VaR-forecast: [  3.37032639   0.91803463   8.82992516  -5.26546206  -6.95172854
 -12.98424934  -4.07035232 -16.09659276  -7.24953931 -13.50920163
 -13.1438927  -13.25714044] 

LLY-1m-loss: 1.7478643485190049 

LLY-1m-VaR-forecast: [-8.37916998 -1.4815798   0.09511685  4.94421541 -7.70725714 -9.84046692
 -1.6399168   0.93342853 -8.04841266  0.67033021 -7.64316883 -2.36775888] 

MRK-1m-loss: 0.6775421493440499 

MRK-1m-VaR-forecast: [ -4.13695358 -18.66440061 -22.10073919  -3.24794958  -7.30067813
 -10.66234938 -23.27108048 -13.31109346 -14.52088368  -5.12156205
  -3.72965318 -12.81150269] 



...

NVO-1m-loss: 0.7316751689588057 

NVO-1m-VaR-forecast: [-13.94838551 -13.17737452 -11.75927964 -17.31611138  -2.14032091
  -5.6686049  -20.64336122 -23.82618425  -5.09342573   1.00514371
  -5.54209451 -15.35681532] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-1m-loss: 1.500690054653104 

NVS-1m-VaR-forecast: [ -1.53477542  -4.41670162  -0.59798413  -4.2965583  -10.94808271
 -11.15681327  -3.92001253  -9.7571145   -6.65483055  -8.79170685
  -9.43009995 -16.78008953] 

PFE-1m-loss: 1.8012508922196033 

PFE-1m-VaR-forecast: [ -0.14624336  -7.91821671  -0.65558583 -16.2738069   -8.05322941
  -8.22203572  -4.02089354  -5.11708409 -15.31130981 -11.37201983
  -6.77635669 -15.72792169] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 4.663419770167789 

ROG-1m-VaR-forecast: [  3.19609747 -32.34829619 -46.75804839 -17.44972878 -18.85442165
 -19.46996605  -9.89040781   7.91281354 -29.6618781  -29.37429799
 -18.58943582   6.21816457] 



### 3-step-ahead VaR: 3 months

In [30]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=3)
    print(f'{t}-3m-loss: {loss} \n')
    print(f'{t}-3m-VaR-forecast: {var_forecast} \n')

AZN-3m-loss: 1.4931088634313388 

AZN-3m-VaR-forecast: [ -7.61857227  -0.37746816  -6.82327351  -0.37745581  -6.82325683
 -11.59327864  -6.82325599 -11.59327851 -12.2343114  -11.48124416
 -12.27998624  -5.46480588 -12.30741192  -5.50104157  -2.12183846
  -5.26026587  -2.06250322 -14.55240479  -1.93132624 -14.50824534
 -17.25928204 -14.50824726 -17.25928576  -1.3498097  -17.25928955
  -1.34981643  -9.01061435  -1.25552064  -8.98265356 -19.6594751 ] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-3m-loss: 1.9476084892670174 

BMY-3m-VaR-forecast: [-10.35690661 -13.34201936 -10.02604033 -12.32950473 -11.13101793
   6.07583502 -10.04379576   3.44236357 -14.01888785   2.70476258
 -14.4061538  -15.90062215 -14.22136722 -18.0323     -19.89720659
 -18.03919733 -20.09181888 -14.45579883 -20.05837988 -14.40407407
  -4.65463676 -13.32710257  -2.13020078 -11.29192142  -3.68305233
 -13.14375537 -18.93910149 -13.64784223 -19.1070124   -5.89287885] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-3m-loss: 6.520812879800585 

JNJ-3m-VaR-forecast: [  3.37032639   1.74190453   7.10402377   0.91803463   7.58958286
  -4.9087387    8.82992516  -7.45970375  -6.03870955  -5.26546206
  -5.82784468 -12.98425197  -6.95172854 -12.99888705  -3.94544182
 -12.98424934  -4.54793072 -15.38173261  -4.07035232 -15.37694417
  -7.1526321  -16.09659276  -6.53277845 -12.95826439  -7.24953931
 -14.74318838 -13.00838563 -13.50920163 -13.18803171 -13.37981351] 

LLY-3m-loss: 5.561117256114587 

LLY-3m-VaR-forecast: [ -8.37916998  -3.14656889   0.04177761  -1.4815798   -0.57106361
   6.85172676   0.09511685   7.26864327  -5.04285842   4.94421541
  -7.70724857  -9.98582483  -7.70725714  -9.98583225  -1.48977672
  -9.84046692  -1.63990757   0.9334401   -1.6399168    0.93342736
 -10.24895785   0.93342853 -10.24899161   0.91720628  -8.04841266
   0.30550679  -7.75103211   0.67033021  -7.6923056   -3.14811929] 

MRK-3m-loss: 2.179476546433824 

MRK-3m-VaR-forecast: [ -4.13695358 -18.66439909 -22.33990173 

...

NVO-3m-loss: 2.233114946831889 

NVO-3m-VaR-forecast: [-13.94838551 -12.99318874  -9.38911658 -13.17737452 -11.29078529
 -19.15370253 -11.75927964 -17.28218922  -3.78356226 -17.31611138
  -2.86070845  -4.45775621  -2.14032091  -5.66860637 -20.88720329
  -5.6686049  -20.88718599 -20.00873588 -20.64336122 -22.24157606
  -6.55626888 -23.82618425  -7.03033727  -1.09914016  -5.09342573
   0.8923506   -5.80181618   1.00514371  -5.68962827 -15.38230973] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-3m-loss: 3.99963114835223 

NVS-3m-VaR-forecast: [ -1.53477542  -3.0736203    0.32941828  -4.41670162  -0.59797584
  -9.07244404  -0.59798413  -9.07240896 -13.16413573  -4.2965583
 -10.01077761 -11.67059964 -10.94808271 -11.19266557  -3.51299286
 -11.15681327  -3.7316416  -11.02465349  -3.92001253 -10.89489725
  -6.48986113  -9.7571145   -6.56340766  -8.26815761  -6.65483055
  -8.43739322  -9.51544557  -8.79170685  -9.44216655 -17.31322722] 

PFE-3m-loss: 6.344665539415022 

PFE-3m-VaR-forecast: [ -0.14624336  -7.74777612  -0.80864032  -7.91821671  -0.65558064
 -19.07902805  -0.65558583 -19.07901339  -6.63397819 -16.2738069
  -6.78046875  -8.19034565  -8.05322941  -8.20173725  -3.61449308
  -8.22203572  -4.02089475  -4.89745285  -4.02089354  -4.89745316
 -17.25898598  -5.11708409 -17.5909877  -11.93584595 -15.31130981
 -11.37346066  -7.13470715 -11.37201983  -6.77637052 -15.72791699] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-3m-loss: 14.067265253501905 

ROG-3m-VaR-forecast: [  3.19609747 -30.10717859 -48.59053532 -32.34829619 -46.7580344
 -17.550956   -46.75804839 -17.55093195 -18.74404524 -17.44972878
 -18.85438908 -19.46998239 -18.85442165 -19.47002957  -2.8406381
 -19.46996605  -2.84095194  -3.9757574   -9.89040781   6.68842756
 -26.34415959   7.91281354 -31.02238488 -29.4055971  -29.6618781
 -29.00663284 -17.94075958 -29.37429799 -17.79262145   6.21818529] 



### 6-step-ahead VaR: 6 months

In [31]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=6)
    print(f'{t}-6m-loss: {loss} \n')
    print(f'{t}-6m-VaR-forecast: {var_forecast} \n')

AZN-6m-loss: 3.1706810928910714 

AZN-6m-VaR-forecast: [ -7.61857227  -0.37746816  -6.82327351 -11.59325879 -12.2343184
  -5.32547297  -0.37745581  -6.82325683 -11.59327864 -12.23431193
  -5.32546774  -1.78551199  -6.82325599 -11.59327851 -12.2343114
  -5.32546617  -1.78551119 -14.49058203 -11.48124416 -12.27998624
  -5.46480588  -2.1264513  -14.60398258 -17.14370876 -12.30741192
  -5.50104157  -2.12183846 -14.62009244 -17.15767145  -1.31984273
  -5.26026587  -2.06250322 -14.55240479 -17.14355542  -1.24890657
  -8.95704015  -1.93132624 -14.50824534 -17.25928204  -1.34980763
  -9.01061341 -19.59687661] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-6m-loss: 3.803834174277333 

BMY-6m-VaR-forecast: [-10.35690661 -13.34201936 -10.02604033   6.16369155 -14.26555839
 -17.67479856 -12.32950473 -11.13101793   6.07583502 -14.28431642
 -16.87330419 -19.86564139 -10.04379576   3.44236357 -14.01888785
 -16.13977158 -18.93604878 -13.74067443   2.70476258 -14.4061538
 -15.90062215 -19.15909797 -14.02491971  -3.59111655 -14.22136722
 -18.0323     -19.89720659 -14.46396087  -4.68281502 -11.77559503
 -18.03919733 -20.09181888 -14.45579883  -4.69799585 -11.79327335
 -19.6583657  -20.05837988 -14.40407407  -4.65463676 -11.78845902
 -19.62905236  -8.1807429 ] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-6m-loss: 10.036195802355255 

JNJ-6m-VaR-forecast: [  3.37032639   1.74190453   7.10402377  -6.98077817  -5.78347205
  -6.97577252   0.91803463   7.58958286  -4.9087387   -6.34832982
  -9.43597798  -4.94737801   8.82992516  -7.45970375  -6.03870955
  -8.36148834  -5.5234706  -19.38537564  -5.26546206  -5.82784468
 -12.98425197  -4.54793521 -15.3817235   -8.20739597  -6.95172854
 -12.99888705  -3.94544182 -16.2130787   -7.62668464 -12.65401393
 -12.98424934  -4.54793072 -15.38173261  -8.20739528 -12.42323774
 -12.22575849  -4.07035232 -15.37694417  -7.1526321  -12.75241454
 -13.19182072 -14.07510976] 

LLY-6m-loss: 10.248203124968898 

LLY-6m-VaR-forecast: [ -8.37916998  -3.14656889   0.04177761   6.34397078  -8.12064047
 -10.9403455   -1.4815798   -0.57106361   6.85172676  -6.23888939
  -9.77139782   0.57314039   0.09511685   7.26864327  -5.04285842
  -8.71553241  -0.14330402  -0.63924277   4.94421541  -7.70724857
  -9.98582483  -1.48979264   0.78315441 -10.57763856  -7.70725714
  

...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-6m-loss: 6.946680125697723 

NVS-6m-VaR-forecast: [ -1.53477542  -3.0736203    0.32941828  -7.41743231 -12.12776206
  -6.39711143  -4.41670162  -0.59797584  -9.07244404 -13.16415085
  -7.44784628  -0.20183797  -0.59798413  -9.07240896 -13.16413573
  -7.44784409  -0.20184563 -14.76240867  -4.2965583  -10.01077761
 -11.67059964  -3.57452574 -10.1540729   -5.53351886 -10.94808271
 -11.19266557  -3.51299286 -10.82283319  -5.89533421  -8.19466688
 -11.15681327  -3.7316416  -11.02465349  -6.50400529  -8.40230136
  -9.33990919  -3.92001253 -10.89489725  -6.48986113  -8.41613013
  -9.33886256 -17.30881489] 

PFE-6m-loss: 11.194358862040271 

PFE-6m-VaR-forecast: [ -0.14624336  -7.74777612  -0.80864032 -18.8657651   -6.91221673
  -5.27220739  -7.91821671  -0.65558064 -19.07902805  -6.63396951
  -5.52401231 -10.70669235  -0.65558583 -19.07901339  -6.63397819
  -5.52401309 -10.7066876  -13.7271974  -16.2738069   -6.78046875
  -8.19034565  -4.36300124  -5.3291671  -17.0612012   -8.05322941
  -

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-6m-loss: 34.81261107730648 

ROG-6m-VaR-forecast: [  3.19609747 -30.10717859 -48.59053532 -16.00163591 -15.69090387
 -18.99388144 -32.34829619 -46.7580344  -17.550956   -18.74402545
 -19.3075679   -3.65140049 -46.75804839 -17.55093195 -18.74404524
 -19.30756918  -3.65138027  -4.53347199 -17.44972878 -18.85438908
 -19.46998239  -2.8408678   -3.97569979 -25.86694805 -18.85442165
 -19.47002957  -2.8406381   -3.97554508 -25.86699049 -33.59593264
 -19.46996605  -2.84095194  -3.9757574  -25.86693252 -33.59612978
 -15.33259608  -9.89040781   6.68842756 -26.34415959 -30.58463507
 -18.95580743   2.56943452] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


### 9-step-ahead VaR: 9 months

In [32]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=9)
    print(f'{t}-9m-loss: {loss} \n')
    print(f'{t}-9m-VaR-forecast: {var_forecast} \n')

AZN-9m-loss: 4.609825204267898 

AZN-9m-VaR-forecast: [ -7.61857227  -0.37746816  -6.82327351 -11.59325879 -12.2343184
  -5.32547297  -1.7855381  -14.49056946 -17.4469945   -0.37745581
  -6.82325683 -11.59327864 -12.23431193  -5.32546774  -1.78551199
 -14.49058154 -17.44699762  -1.56948889  -6.82325599 -11.59327851
 -12.2343114   -5.32546617  -1.78551119 -14.49058203 -17.44699746
  -1.56948676  -9.04659518 -11.48124416 -12.27998624  -5.46480588
  -2.1264513  -14.60398258 -17.14370876  -1.30266056  -8.90837723
 -19.60122419] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-9m-loss: 5.481814508810399 

BMY-9m-VaR-forecast: [-10.35690661 -13.34201936 -10.02604033   6.16369155 -14.26555839
 -17.67479856 -20.9293687  -13.49234615  -4.66694447 -12.32950473
 -11.13101793   6.07583502 -14.28431642 -16.87330419 -19.86564139
 -13.47942501  -3.92777867 -12.45338649 -10.04379576   3.44236357
 -14.01888785 -16.13977158 -18.93604878 -13.74067443  -4.1488138
 -11.84702519 -19.51679855   2.70476258 -14.4061538  -15.90062215
 -19.15909797 -14.02491971  -3.59111655 -11.32417218 -19.3802473
  -7.79808906] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-9m-loss: 18.656016345856703 

JNJ-9m-VaR-forecast: [  3.37032639   1.74190453   7.10402377  -6.98077817  -5.78347205
  -6.97577252  -3.6057269  -16.04913979  -3.51126687   0.91803463
   7.58958286  -4.9087387   -6.34832982  -9.43597798  -4.94737801
 -16.5141423   -3.76902231  -9.20223486   8.82992516  -7.45970375
  -6.03870955  -8.36148834  -5.5234706  -19.38537564  -2.84206527
  -8.04418116 -11.40287586  -5.26546206  -5.82784468 -12.98425197
  -4.54793521 -15.3817235   -8.20739597 -12.423243   -12.22575706
 -13.42652341] 

LLY-9m-loss: 15.78292993255084 

LLY-9m-VaR-forecast: [ -8.37916998  -3.14656889   0.04177761   6.34397078  -8.12064047
 -10.9403455    2.21509791  -2.493857    -9.66265944  -1.4815798
  -0.57106361   6.85172676  -6.23888939  -9.77139782   0.57314039
  -1.14579397  -8.57828052   3.57999185   0.09511685   7.26864327
  -5.04285842  -8.71553241  -0.14330402  -0.63924277  -7.87879992
   4.00405111  -7.21439137   4.94421541  -7.70724857  -9.98582483
  -1.48979264   0

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-9m-loss: 7.0737622210069535 

NVO-9m-VaR-forecast: [-13.94838551 -12.99318874  -9.38911658 -19.98714691  -4.05150133
  -2.1188358  -19.51696728 -23.78985585  -7.2883081  -13.17737452
 -11.29078529 -19.15370253  -4.35810183  -3.98943234 -19.82126931
 -26.02544916  -7.90842839  -1.13433981 -11.75927964 -17.28218922
  -3.78356226  -4.94322799 -20.76918518 -25.24826307  -7.05158821
  -0.19877484  -6.73997139 -17.31611138  -2.86070845  -4.45775621
 -20.84780068 -21.69927395  -5.61926668   1.00251622  -5.72194365
 -17.29205646] 

NVS-9m-loss: 13.498143000091588 

NVS-9m-VaR-forecast: [ -1.53477542  -3.0736203    0.32941828  -7.41743231 -12.12776206
  -6.39711143  -2.06521684 -16.53704103  -1.50036463  -4.41670162
  -0.59797584  -9.07244404 -13.16415085  -7.44784628  -0.20183797
 -14.76244026  -3.81938616  -7.60377843  -0.59798413  -9.07240896
 -13.16413573  -7.44784409  -0.20184563 -14.76240867  -3.8193871
  -7.60376893  -8.4465692   -4.2965583  -10.01077761 -11.67059964
  -3.57452574 -1

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


### 12-step-ahead VaR: 12 months

In [33]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=12)
    print(f'{t}-12m-loss: {loss} \n')
    print(f'{t}-12m-VaR-forecast: {var_forecast} \n')

AZN-12m-loss: 6.112571314373784 

AZN-12m-VaR-forecast: [ -7.61857227  -0.37746816  -6.82327351 -11.59325879 -12.2343184
  -5.32547297  -1.7855381  -14.49056946 -17.4469945   -1.56948678
  -9.04659041 -19.57056116] 

BMY-12m-loss: 6.977228561615677 

BMY-12m-VaR-forecast: [-10.35690661 -13.34201936 -10.02604033   6.16369155 -14.26555839
 -17.67479856 -20.9293687  -13.49234615  -4.66694447 -11.60284756
 -18.84593137  -7.24358276] 

JNJ-12m-loss: 27.599794656674657 

JNJ-12m-VaR-forecast: [  3.37032639   1.74190453   7.10402377  -6.98077817  -5.78347205
  -6.97577252  -3.6057269  -16.04913979  -3.51126687 -10.48410758
 -12.60767025 -15.85444335] 

LLY-12m-loss: 19.316729795142848 

LLY-12m-VaR-forecast: [ -8.37916998  -3.14656889   0.04177761   6.34397078  -8.12064047
 -10.9403455    2.21509791  -2.493857    -9.66265944   2.97112399
  -7.13702649  -1.97438552] 

MRK-12m-loss: 8.451495756064968 

MRK-12m-VaR-forecast: [ -4.13695358 -18.66439909 -22.33990173  -4.60239156  -6.32411118
 -11.

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-12m-loss: 9.065536592175462 

NVO-12m-VaR-forecast: [-13.94838551 -12.99318874  -9.38911658 -19.98714691  -4.05150133
  -2.1188358  -19.51696728 -23.78985585  -7.2883081   -1.80944787
  -6.14508062 -18.13767089] 

NVS-12m-loss: 22.366612949619462 

NVS-12m-VaR-forecast: [ -1.53477542  -3.0736203    0.32941828  -7.41743231 -12.12776206
  -6.39711143  -2.06521684 -16.53704103  -1.50036463  -5.35538825
  -7.5547542  -16.67046709] 

PFE-12m-loss: 13.253880571239803 

PFE-12m-VaR-forecast: [ -0.14624336  -7.74777612  -0.80864032 -18.8657651   -6.91221673
  -5.27220739 -10.5699475  -13.72432597  -9.3393778   -7.67755946
  -7.5958283  -18.91814021] 

ROG-12m-loss: 55.55945194718216 

ROG-12m-VaR-forecast: [  3.19609747 -30.10717859 -48.59053532 -16.00163591 -15.69090387
 -18.99388144  -5.96412594   1.88212676 -24.40205621 -33.35326101
 -14.58792684   5.75084303] 



### ABBV case

In [34]:
# ABBV case
data = pd.read_csv('non_nan.csv', )
data2 = pd.read_csv('monthly_data.csv')
data_ABBV = data2[['Date', 'ABBV']]
data_ABBV = data_ABBV.dropna()
data_ABBV

,Date,ABBV
158,2013-03-01,10.455062
159,2013-04-01,12.922975
160,2013-05-01,-6.438455
161,2013-06-01,-3.162299
162,2013-07-01,10.014486
...,...,...
276,2023-01-01,-8.576205
277,2023-02-01,5.152609
278,2023-03-01,3.554254
279,2023-04-01,-5.176628


In [35]:
merged_df = data_ABBV.merge(data, on='Date', how='inner')
merged_df

,Date,ABBV,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2013-03-01,10.455062,7.5,232.282,1.984211,0.397212,3.337507,14.582205,11.414669,7.978126,4.851792,3.440210,-7.714279,9.067089,5.443922,-0.188640,-0.655,-0.2,0.069449
1,2013-04-01,12.922975,7.6,231.797,1.957500,0.085246,2.374537,3.881563,-3.568848,4.538222,-2.482836,7.359867,11.497419,3.537347,0.727651,-10.457790,-0.485,0.1,-0.026711
2,2013-05-01,-6.438455,7.5,231.893,1.759091,-0.190605,2.361004,-1.290451,16.816781,-1.231989,-4.008663,-0.638313,-8.792401,-2.711502,-6.329562,8.513136,0.096,-0.1,-0.198409
3,2013-06-01,-3.162299,7.5,232.445,1.928182,0.620501,-1.853781,-7.707318,-2.868894,2.762464,-6.788137,-0.535372,-3.805087,-1.463219,3.722003,2.269287,0.552,0.0,0.169091
4,2013-07-01,10.014486,7.3,232.900,2.300000,0.210359,5.724173,7.230424,-3.244595,8.898141,8.122938,4.655361,9.001745,1.272809,4.355584,17.540151,0.455,-0.2,0.371818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023-01-01,-8.576205,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
119,2023-02-01,5.152609,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
120,2023-03-01,3.554254,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
121,2023-04-01,-5.176628,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [36]:
merged_df.columns

Index(['Date', 'ABBV', 'unemployment_rate', 'CPI', 'treasury_yield',
       'GDP_growth', 'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO',
       'NVS', 'PFE', 'ROG', 'inflation_change', 'unemp_change',
       'treasury_yield_change'],
      dtype='object')

In [37]:
# Creating lagged variables
tickers_plus_ABBV = ['ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

merged_df, lagged_covs = create_lags_cov(merged_df, covs, period=period_covs)
merged_df, lagged_tickers = create_lags_returns(merged_df, tickers_plus_ABBV, period= period_returns)

In [38]:
# Creating dummy variables
merged_df['Quater_dummy'] = 0
merged_df['Quater_rippel'] = 0
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
for index, row in merged_df.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        merged_df.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        merged_df.loc[index, 'Quater_dummy'] = 1

In [39]:
merged_df.columns

Index(['Date', 'ABBV', 'unemployment_rate', 'CPI', 'treasury_yield',
       'GDP_growth', 'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO',
       'NVS', 'PFE', 'ROG', 'inflation_change', 'unemp_change',
       'treasury_yield_change', 'unemployment_rate_lag1',
       'unemployment_rate_lag2', 'unemployment_rate_lag3', 'CPI_lag1',
       'CPI_lag2', 'CPI_lag3', 'treasury_yield_lag1', 'treasury_yield_lag2',
       'treasury_yield_lag3', 'GDP_growth_lag1', 'GDP_growth_lag2',
       'GDP_growth_lag3', 'SP500_return_lag1', 'SP500_return_lag2',
       'SP500_return_lag3', 'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'ABBV_lag1',
       'ABBV_lag2', 'AZN_lag1', 'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1',
       'JNJ_lag2', 'LLY_lag1', 'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1

## 1. Macros Only

In [40]:
to_exclude = []

for t in tickers_plus_ABBV:
    to_exclude.append(t)
    to_exclude.append(f'{t}_lag1')
    to_exclude.append(f'{t}_lag2')

for covariate in covs:
    to_exclude.append(covariate)

to_exclude.append('Date')

In [41]:
to_exclude

['ABBV',
 'ABBV_lag1',
 'ABBV_lag2',
 'AZN',
 'AZN_lag1',
 'AZN_lag2',
 'BMY',
 'BMY_lag1',
 'BMY_lag2',
 'JNJ',
 'JNJ_lag1',
 'JNJ_lag2',
 'LLY',
 'LLY_lag1',
 'LLY_lag2',
 'MRK',
 'MRK_lag1',
 'MRK_lag2',
 'NVO',
 'NVO_lag1',
 'NVO_lag2',
 'NVS',
 'NVS_lag1',
 'NVS_lag2',
 'PFE',
 'PFE_lag1',
 'PFE_lag2',
 'ROG',
 'ROG_lag1',
 'ROG_lag2',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'Date']

In [43]:
# 1 month
loss, var_forecast = rolling_forecast_QR(merged_df, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=1)
print('ABBV-1m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')

# 3 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=3)
print('ABBV-3m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')


# 6 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=6)
print('ABBV-6m-loss: {loss} \n')
print('ABBV-VaR-forecast:{var_forecast} \n')

# 9 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=9)
print('ABBV-9m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')


# 12 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=12)
print('ABBV-12m-loss: {loss} \n')
print('ABBV-VaR-forecast:{var_forecast} \n')

ABBV-1m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-3m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-6m-loss: {loss} 

ABBV-VaR-forecast:{var_forecast} 

ABBV-9m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-12m-loss: {loss} 

ABBV-VaR-forecast:{var_forecast} 



## 2. Ticker data

In [44]:
merged_df_stocks = merged_df.copy()
to_exclude2 = []

for covariate in covs:
    to_exclude2.append(covariate)
    to_exclude2.append(f'{covariate}_lag1')
    to_exclude2.append(f'{covariate}_lag2')
    to_exclude2.append(f'{covariate}_lag3')

for t in tickers:
    to_exclude2.append(t)

to_exclude2.append('Date')

In [45]:
# 1 month
loss, var_forecast = rolling_forecast_QR(merged_df, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=1)
print('ABBV-1m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')

# 3 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=3)
print('ABBV-3m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')


# 6 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=6)
print('ABBV-6m-loss: {loss} \n')
print('ABBV-VaR-forecast:{var_forecast} \n')

# 9 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=9)
print('ABBV-9m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')


# 12 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=12)
print('ABBV-12m-loss: {loss} \n')
print('ABBV-VaR-forecast:{var_forecast} \n')

ABBV-1m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-3m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-6m-loss: {loss} 

ABBV-VaR-forecast:{var_forecast} 

ABBV-9m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-12m-loss: {loss} 

ABBV-VaR-forecast:{var_forecast} 



## 3. All Covariates

In [46]:
merged_df_covs = merged_df.copy()
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

In [47]:
# 1 month
loss, var_forecast = rolling_forecast_QR(merged_df, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=1)
print('ABBV-1m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')

# 3 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=3)
print('ABBV-3m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')


# 6 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=6)
print('ABBV-6m-loss: {loss} \n')
print('ABBV-VaR-forecast:{var_forecast} \n')

# 9 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=9)
print('ABBV-9m-loss: {loss} \n')
print('ABBV-VaR-forecast: {var_forecast} \n')


# 12 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=12)
print('ABBV-12m-loss: {loss} \n')
print('ABBV-VaR-forecast:{var_forecast} \n')

ABBV-1m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-3m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-6m-loss: {loss} 

ABBV-VaR-forecast:{var_forecast} 

ABBV-9m-loss: {loss} 

ABBV-VaR-forecast: {var_forecast} 

ABBV-12m-loss: {loss} 

ABBV-VaR-forecast:{var_forecast} 

